In [963]:
import numpy as np

def padded_binary(i: int, n: int) -> str:
    return bin(i)[2:].zfill(n)

def binary_string_to_array(string: str):
    return np.array(list(string), dtype=int)

def calculate_valid_transitions(n: int):
    num_states = 2**n
    all_states = [padded_binary(i, n) for i in range(num_states)]
    
    # print(f"Total number of states: {num_states}")
    print("Valid single-step transitions:")

    valid_difference_vectors = set()
    
    valid_E_reactions = [] # distributively
    valid_F_reactions = [] # distributively

    for i in range(num_states):
        for j in range(num_states):
            # Do not consider transitions from a state to itself
            if i == j:
                continue

            if np.sum(np.abs(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]))) == 1:
                # Determine if it's a phosphorylation or dephosphorylation event
                # A +1 indicates phosphorylation, a -1 indicates dephosphorylation
                element = "E" if np.any(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]) == 1) else "F"
                
                if element == "E":
                    print(f"{all_states[i]} --> {all_states[j]} ({element}), {i}, {j}")
                    valid_E_reactions.append([all_states[i], all_states[j], i, j, element])
                if element == "F":
                    print(f"{all_states[i]} --> {all_states[j]} ({element}), {i}, {j}")
                    valid_F_reactions.append([all_states[i], all_states[j], i, j, element])

                # Add the difference vector to a set to find unique vectors
                valid_difference_vectors.add(tuple(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i])))
                
    # for vec in sorted(list(valid_difference_vectors)):
    #     print(f"{list(vec)}")

    return valid_E_reactions, valid_F_reactions
# Run the function for a system of 3 proteins
n = 1
valid_E_reactions, valid_F_reactions = calculate_valid_transitions(n)
print("Valid E reactions are: ", valid_E_reactions, "with total number of reactions = ", len(valid_E_reactions))
print("Valid F reactions are: ", valid_F_reactions, "with total number of reactions = ", len(valid_F_reactions))


Valid single-step transitions:
0 --> 1 (E), 0, 1
1 --> 0 (F), 1, 0
Valid E reactions are:  [['0', '1', 0, 1, 'E']] with total number of reactions =  1
Valid F reactions are:  [['1', '0', 1, 0, 'F']] with total number of reactions =  1


In [ ]:
import numpy as np
import sympy as sp
from typing import Dict

def generate_odes(n: int) -> Dict[sp.Symbol, sp.Expr]:
    if n <= 0:
        raise ValueError("n must be a positive integer")

    valid_E_reactions, valid_F_reactions = calculate_valid_transitions(n)
    num_states = 2**n  

    # symbolic objects
    E = sp.Symbol('E'); F = sp.Symbol('F')
    S = sp.IndexedBase('S')
    ES = sp.IndexedBase('ES'); EdotS = sp.IndexedBase('E.S')
    FS = sp.IndexedBase('FS'); FdotS = sp.IndexedBase('F.S')

    a_E = sp.IndexedBase('a^E'); a_F = sp.IndexedBase('a^F')
    b_E = sp.IndexedBase('b^E'); b_F = sp.IndexedBase('b^F')
    c_E = sp.IndexedBase('c^E'); c_F = sp.IndexedBase('c^F')

    d_dt = {}

    # Build maps:
    # outgoing: k -> [j]
    cE_out = {}
    for _, _, k, j, _ in valid_E_reactions:
        cE_out.setdefault(k, []).append(j)
    cF_out = {}
    for _, _, k, j, _ in valid_F_reactions:
        cF_out.setdefault(k, []).append(j)

    cE_in = {}
    for _, _, k, j, _ in valid_E_reactions:
        cE_in.setdefault(j, []).append(k)
    cF_in = {}
    for _, _, k, j, _ in valid_F_reactions:
        cF_in.setdefault(j, []).append(k)

    print("cE_out_map:", cE_out)
    print("cF_out_map:", cF_out)
    print("cE_in_map:", cE_in)
    print("cF_in_map:", cF_in) 

    for i in range(num_states):

        term_E = 0
        if 0 <= i < N:
            term_E += b_E[i] * ES[i]
            term_E -= a_E[i] * EdotS[i]
        else:
            term_E += 0

        term_F = 0
        if 0 < i <= N:
            term_F += b_F[i] * FS[i]
            term_F -= a_F[i] * FdotS[i]

        sum_cE = sum(c_E[k, i] * ES[k] for k in cE_in.get(i, [])) if i in cE_in else 0
        sum_cF = sum(c_F[k, i] * FS[k] for k in cF_in.get(i, [])) if i in cF_in else 0

        d_dt[S[i]] = term_E + term_F + sum_cE + sum_cF

    for i in range(0, num_states - 1):
        sum_c_E_ij = sum(c_E[i, j] for j in cE_out.get(i, [])) if i in cE_out else 0
        d_dt[ES[i]] = a_E[i] * EdotS[i] - (b_E[i] + sum_c_E_ij) * ES[i]

    for i in range(1, num_states):
        sum_c_F_ij = sum(c_F[i, j] for j in cF_out.get(i, [])) if i in cF_out else 0
        d_dt[FS[i]] = a_F[i] * FdotS[i] - (b_F[i] + sum_c_F_ij) * FS[i]

    d_dt[E] = sum(
        -a_E[j] * EdotS[j]
        + (b_E[j] + (sum(c_E[j, k] for k in cE_out.get(j, [])) if j in cE_out else 0)) * ES[j]
        for j in range(0, num_states - 1)  
    )

    d_dt[F] = sum(
        -a_F[j] * FdotS[j]
        + (b_F[j] + (sum(c_F[j, k] for k in cF_out.get(j, [])) if j in cF_out else 0)) * FS[j]
        for j in range(1, num_states)  
    )
    # for i in len(d_dt[ES]):
    #     print(i)
    return d_dt

n = 1
# valid_E_reactions, valid_F_reactions = calculate_valid_transitions(n)
# print("Valid E reactions are: ", valid_E_reactions, "with total number of reactions = ", len(valid_E_reactions))
# print("Valid F reactions are: ", valid_F_reactions, "with total number of reactions = ", len(valid_F_reactions))
print()
odes = generate_odes(n)
for k, v in odes.items():
    print(k, "->", v)

# print(type(odes))


Valid single-step transitions:
0 --> 1 (E), 0, 1
1 --> 0 (F), 1, 0
cE_out_map: {0: [1]}
cF_out_map: {1: [0]}
cE_in_map: {1: [0]}
cF_in_map: {0: [1]}
S[0] -> -E.S[0]*a^E[0] + ES[0]*b^E[0] + FS[1]*c^F[1, 0]
S[1] -> -E.S[1]*a^E[1] + ES[0]*c^E[0, 1] + ES[1]*b^E[1] - F.S[1]*a^F[1] + FS[1]*b^F[1]
ES[0] -> -(b^E[0] + c^E[0, 1])*ES[0] + E.S[0]*a^E[0]
FS[1] -> -(b^F[1] + c^F[1, 0])*FS[1] + F.S[1]*a^F[1]
E -> (b^E[0] + c^E[0, 1])*ES[0] - E.S[0]*a^E[0]
F -> (b^F[1] + c^F[1, 0])*FS[1] - F.S[1]*a^F[1]
<class 'dict'>


In [977]:
odes[S[0]] + odes[S[1]] + odes[ES[0]]

-(b^E[0] + c^E[0, 1])*ES[0] - E.S[1]*a^E[1] + ES[0]*b^E[0] + ES[0]*c^E[0, 1] + ES[1]*b^E[1] - F.S[1]*a^F[1] + FS[1]*b^F[1] + FS[1]*c^F[1, 0]

In [978]:
odes[ES[0]]+ odes[E]

0

In [979]:
odes[FS[1]] + odes[F]

0

In [ ]:
# def defining_phosphorylation_reactions(n: int, phosphorylation_dictionary: dict) -> list:

_IncompleteInputError: incomplete input (944706081.py, line 1)